In [20]:
import requests
import pandas as pd
import sqlite3
import logging
import threading

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('dados_pontos.db')

# Criar tabela (se ainda não existir)
conn.execute('''
    CREATE TABLE IF NOT EXISTS onibus (
        NOME TEXT,
        NUM TEXT,
        Latitude REAL,
        Longitude REAL,
        SEQ INTEGER,
        GRUPO TEXT,
        SENTIDO TEXT,
        TIPO TEXT,
        ID_do_Itinerario INTEGER,
        COD TEXT
    )
''')
conn.commit()

# Função para buscar e processar dados da linha de ônibus
def buscar_e_processar_dados(linha):
    url = f'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={linha:03}&c=821f0'
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        dados_json = response.json()

        # Verificar se os dados estão vazios
        if not dados_json:
            logging.info(f"Linha {linha}: Nenhum dado encontrado.")
            return None

        # Criar DataFrame a partir dos dados
        df = pd.DataFrame.from_records(dados_json)

        # Substituir vírgulas por pontos e converter tipos
        df['Latitude'] = df['LAT'].str.replace(',', '.').astype(float)
        df['Longitude'] = df['LON'].str.replace(',', '.').astype(float)
        df['SEQ'] = df['SEQ'].astype(int)
        df['ID_do_Itinerario'] = df['ITINERARY_ID'].astype(int)
        
        # Adicionar coluna COD
        df['COD'] = linha

        return df[['NOME', 'NUM', 'Latitude', 'Longitude', 'SEQ', 'GRUPO', 'SENTIDO', 'TIPO', 'ID_do_Itinerario', 'COD']]

    except requests.RequestException as e:
        logging.error(f"Erro na requisição para a linha {linha}: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar a linha {linha}: {e}")
    return None

# Função para processar cada linha e armazenar os resultados em uma lista com Lock
def processar_linha(linha, result_list, lock):
    df_result = buscar_e_processar_dados(linha)
    if df_result is not None:
        with lock:  # Garantir acesso seguro à lista
            result_list.append(df_result)
            logging.info(f"Linha {linha} adicionada com sucesso.")

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    df_result.to_sql('onibus', conn, if_exists='append', index=False)
    logging.info("Dados gravados no banco de dados.")

# Função para gerenciar threads e processamento de dados com limite de 5 threads
def processar_linhas_com_threads(linhas_unicas):
    result_list = []
    lock = threading.Lock()  # Trava para garantir acesso seguro à lista
    max_threads = 5

    # Dividir as linhas em grupos de 5 threads
    for i in range(0, len(linhas_unicas), max_threads):
        threads = []
        for linha in linhas_unicas[i:i + max_threads]:
            thread = threading.Thread(target=processar_linha, args=(linha, result_list, lock))
            threads.append(thread)
            thread.start()

        # Esperar todas as threads do grupo atual terminarem
        for thread in threads:
            thread.join()

    # Verificar e concatenar resultados
    if result_list:
        df_concatenado = pd.concat(result_list, ignore_index=True)
        logging.info(f"Total de linhas processadas: {len(df_concatenado)}")
        gravar_no_banco(df_concatenado)
    else:
        logging.info("Nenhum dado para gravar no banco.")

# Inicializar o processamento com threads
linhas_unicas = range(0, 100)

try:
    processar_linhas_com_threads(linhas_unicas)
finally:
    # Fechar a conexão com o banco de dados ao final
    conn.close()
    logging.info("Conexão com o banco de dados fechada.")


2024-10-16 22:15:11,362 - INFO - Linha 4: Nenhum dado encontrado.
2024-10-16 22:15:11,363 - INFO - Linha 0: Nenhum dado encontrado.
2024-10-16 22:15:11,364 - INFO - Linha 2: Nenhum dado encontrado.
2024-10-16 22:15:11,364 - INFO - Linha 3: Nenhum dado encontrado.
2024-10-16 22:15:11,365 - INFO - Linha 1: Nenhum dado encontrado.
2024-10-16 22:15:11,770 - INFO - Linha 9: Nenhum dado encontrado.
2024-10-16 22:15:11,771 - INFO - Linha 6: Nenhum dado encontrado.
2024-10-16 22:15:11,772 - INFO - Linha 5: Nenhum dado encontrado.
2024-10-16 22:15:11,774 - INFO - Linha 7: Nenhum dado encontrado.
2024-10-16 22:15:11,779 - INFO - Linha 8: Nenhum dado encontrado.
2024-10-16 22:15:12,182 - INFO - Linha 13: Nenhum dado encontrado.
2024-10-16 22:15:12,183 - INFO - Linha 12: Nenhum dado encontrado.
2024-10-16 22:15:12,183 - INFO - Linha 14: Nenhum dado encontrado.
2024-10-16 22:15:12,226 - INFO - Linha 11 adicionada com sucesso.
2024-10-16 22:15:12,230 - INFO - Linha 10 adicionada com sucesso.
2024-10

In [21]:
df_concatenado

,NOME,NUM,Latitude,Longitude,SEQ,GRUPO,SENTIDO,TIPO,ID do Itinerario,COD
0,"Rua Antônio Schiebel, 1624 - Boqueirão",150335,-25.500940,-49.242273,10,,Terminal Carmo,Chapéu chinês,737,521
1,"Rua Júlio Zandoná, 755 - Alto Boqueirão",150342,-25.522215,-49.249518,1,,Terminal Carmo,Chapéu chinês,737,521
2,"Rua Cascavel, 328 - Boqueirão",150749,-25.503470,-49.251600,7,,Nivaldo Braga,Chapéu chinês,13833,521
3,"Rua Antônio Schiebel, 1957 - Boqueirão",150784,-25.502059,-49.245364,3,,Nivaldo Braga,Chapéu chinês,13833,521
4,"Rua Max Schubert, 195 - Alto Boqueirão",150340,-25.520898,-49.253236,16,,Nivaldo Braga,Chapéu chinês,13833,521
5,"Rua Jorge Brey, 343 - Xaxim",150897,-25.522123,-49.253694,17,,Nivaldo Braga,Placa em poste,13833,521
6,"Rua Paulo Setúbal, 4103 - Hauer",150044,-25.512567,-49.244580,5,,Terminal Carmo,Chapéu chinês,737,521
7,"Rua Paulo Setúbal, 3896 - Hauer",150045,-25.511051,-49.245313,6,,Terminal Carmo,Domus,737,521
8,"Rua Paulo Setúbal, 3377 -Hauer",150046,-25.506771,-49.247379,7,,Terminal Carmo,Chapéu chinês,737,521
9,"Rua Max Schubert, 495 - Alto Boqueirão",150339,-25.521198,-49.250695,15,,Nivaldo Braga,Chapéu chinês,13833,521
